In [ ]:
import pandas as pd
import numpy as np
POS_DICT = {
    'Goalkeeper': ['GK'],
    'Full Back': ['LB', 'LB5', 'RB', 'RB5'],
    'Wing Back': ['LWB', 'RWB'],
    'Center Back': ['CB', 'CB3', 'RCB', 'LCB', 'RCB3', 'LCB3'],
    'Defensive Midfielder': ['DMF', 'RDMF', 'LDMF'],
    'Central Midfielder': ['LCMF', 'LCMF3', 'RCMF', 'RCMF3'],
    'Attacking Midfielder': ['AMF', 'LAMF', 'RAMF'],
    'Winger': ['LW', 'RW'],
    'Wing Forward': ['LWF', 'RWF'],
    'Striker': ['CF']
}


In [ ]:
file_path = 'C:/Users/githk/Projects/footballindex/Sample Data/raw stats/GRE1.xlsx'

gr_df = pd.read_excel(file_path)

In [ ]:
def column_names_to_lower(df):
    df.columns = [col.lower() for col in gr_df.columns]
    return df

In [ ]:
def per_90_to_absolute(df, min_cols='minutes played'):
    per_90_cols = [col for col in df.columns if '90' in col]
    col_names = list(map(lambda x: x.replace(' per 90', '') + ' extrp', per_90_cols))
    for i, col in enumerate(per_90_cols):
        df[col_names[i]] = df[col] * df[min_cols] / 90
    return df

In [ ]:
def successful_percentage_to_absolute(df, success_cols=['won', 'accurate', 'successful', 'on target', 'conversion']):
    percentage_cols = [col for col in df.columns if '%' in col]
    for col in percentage_cols:
        for keyword in success_cols:
            if keyword in col:
                col_name = col.replace(', %', '') + ' per 90'
                df[col_name] = df[col.replace(keyword, '').replace(', %', '').strip() + ' per 90'] * df[col] / 100
    return df

In [ ]:
def map_positions(df):
    '''
    Maps the abbreviation of the postion.
    '''
    
    def element_wise_mapping(position):
        '''
        Element wise mapping.
        position [str]: The abbreviated position.

        returns [str, int]: The full description of the position or -1 if the position is not included in the dictionary's values.
        '''
        for key in POS_DICT:
            for values in POS_DICT[key]:
                if position in values:
                    return key
        return -1

    # Expands the single position column to as many columns are unique positions separated by commas.
    positions_df = df['position'].str.split(', ', expand=True)
    mapped_positions_df = positions_df.applymap(element_wise_mapping, na_action='ignore')
    # Creates a Dataframe with a single column, named position. Containing only unique mapped positions.
    unique_pos_list_df = mapped_positions_df.stack().groupby(level=0).apply(lambda x: x.unique()).rename('position').to_frame()
    
    df1 = pd.DataFrame(unique_pos_list_df['position'].tolist(), index=unique_pos_list_df.index).add_prefix('position')
    return df1

In [ ]:
gr_df = column_names_to_lower(gr_df)
gr_df.rename(columns={'penalty conversion, %': 'penalties conversion, %'}, inplace=True)

In [ ]:
# Conver minutes played to how many 90s each player has played.

gr_df['90s'] = gr_df['minutes played'].map(lambda val: val / 90 if val else -1)
gr_df[['90s', 'minutes played']].head()

In [ ]:
non_per_90s_cols = ['padj sliding tackles', 'padj interceptions', 'penalties taken']
new_per_90_cols = []
for col in non_per_90s_cols:
    per_90_col = col + ' per 90'
    new_per_90_cols.append(per_90_col)
    gr_df[per_90_col] = gr_df[col] / gr_df['90s']

In [ ]:
gr_df = successful_percentage_to_absolute(gr_df)
gr_df = per_90_to_absolute(gr_df)
gr_df = pd.concat([gr_df, map_positions(gr_df)], axis=1)

In [ ]:
# Create the reatio of assist to xA per 90 mins.

gr_df['assists to xa per 90'] = gr_df['assists per 90'] / gr_df['xa per 90']

In [ ]:
# Create the reatio of assist to xA.

gr_df['assists to xa'] = gr_df['assists per 90'] / gr_df['xa']

In [ ]:
gr_df['minutes per xa'] = gr_df.apply(lambda row: row['minutes played'] / row['xa'] if row['xa'] else -1, axis=1)

In [ ]:
gr_df['minutes per assist'] = gr_df.apply(lambda row: row['minutes played'] / row['assists'] if row['assists'] else -1, axis=1)

In [ ]:
# Get goals to xG per 90 mins, if this ratio is greater than one it means that the player is a clinical finsiher.
gr_df['goals to xg per 90'] = gr_df.apply(lambda row: row['goals per 90'] / row['xg per 90'] if row['xg per 90'] > 0 else -1, axis=1)

In [ ]:
gr_df['goals to xg'] = gr_df.apply(lambda row: row['goals'] / row['xg'] if row['xg'] else -1, axis=1)

In [ ]:
gr_df.rename(columns={'penalty conversion, %': 'penalties conversion, %'}, inplace=True)

[col for col in gr_df.columns if 'convers' in col]

